@Author: Fantasy_Silence            
@Time: 2024-04-24                   
@IDE: Visual Studio Code & PyCharm  
@Python: 3.9.7                      
@Description: Using my data         

# 一、导入必要的模块

In [ ]:
print("=" * 50)
print("1.导入必要的模块")
print("=" * 50)
import time
import pandas as pd
import matplotlib.pyplot as plt

from src.common.infoTool.const import CONST_TABLE
from src.modules.datapreparation.dataparser import HousingDataParser
from src.modules.datapreparation.datacrawler import HousingDataSpider
from src.common.infoTool.randomIPandHeaders import RandomRequestInfoGenerator
print(
    "完成'导入必要的模块'", end="\n\n"
)

# 二、数据爬取与解析

In [ ]:
print("=" * 50)
print("0.数据爬取与解析")
print("=" * 50)
start_time = time.time()
for city in CONST_TABLE["CITY"].keys():
    proxies, headers = RandomRequestInfoGenerator.get()
    HousingDataSpider(city=city, headers=headers, proxies=proxies)
    HousingDataParser(city=city)
end_time = time.time()
print(
    "完成'数据爬取与解析', 用时%.3fms" % 
    ((end_time - start_time) * 1000), end="\n\n"
)